### Weekly_LinzessSnapshot_ProfileInfo_Feed

- FILTER NOTE :
- Xponent_Calls Modified to calc flags for just LI1,LI2,LI3 instead of Fetch Products
- NOBS was matching with presc_profile info before said modification , saw slight redction after.

In [1]:
import polars as pl
import pandas as pd
import gc
import numpy as np
import json

In [2]:
# load variables from JSON
with open('vars_wk.json', 'r') as json_file:
    js = json.load(json_file)

bucket = js['bucket']
data_date = js['data_date']
num_weeks_rx = js['num_weeks_rx']
num_weeks_calls = js['num_weeks_calls']
IBSC_ptype_file = js['IBSC_ptype_file']

dflib = f's3://{bucket}/BIT/dataframes/'
xpn = f's3://{bucket}/PYADM/weekly/archive/{data_date}/xponent/'

In [3]:
# Utility Functions -
def load(df, lib=dflib):
    globals()[df] = pl.read_parquet(f'{lib}{df}.parquet')

In [4]:
# Imporing Dependencies
prod_mapping = pl.read_csv(f's3://{bucket}/BIT/docs/productmapping_pybit.txt',separator='|')
geo_code_mapper = pl.from_pandas(pd.read_excel(f's3://{bucket}/BIT/docs/geo_id_full.xlsx'))
ibsc_ptype = pl.from_pandas(pd.read_excel(f's3://{bucket}/BIT/docs/{IBSC_ptype_file}.xlsx'))
load('mp_spec_seg_dec')
load('MASTER_UNI')
load('temp_calls')
fetch_products = ['LI1','LI2','LI3','TRU','AMT','LAC','MOT','LUB','IRL']

## Generator Functions -

In [5]:
# For Voucher Removal - 
def get_lin_voucher():
    vch = pl.read_parquet(f'{xpn}LIN_VOUCHER.parquet') # n_rows=500
    vch1 = pl.DataFrame()
    for prod in ['LIN1','LIN2','LIN3']: # LINV
        vch_prod = (
            vch.select(
                pl.col('IID'),
                pl.col(f'{prod}TUF1').alias(f'vTUF_1c'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(1,5)]).alias(f'vTUF_4c'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(1,14)]).alias(f'vTUF_13c'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(1,27)]).alias(f'vTUF_26c'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(1,num_weeks_rx+1)]).alias(f'vTUF_qtdc'),
                pl.col(f'{prod}TUF2').alias(f'vTUF_1p'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(5,9)]).alias(f'vTUF_4p'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(14,27)]).alias(f'vTUF_13p'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(27,53)]).alias(f'vTUF_26p'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(14,14+num_weeks_rx)]).alias(f'vTUF_qtdp'),
                pl.sum_horizontal([f'{prod}TUF{i}' for i in range(1,106)]).alias(f'vTUF_all')
            )
            .with_columns(pl.lit(f'LI{prod[-1]}').alias('PROD_CD'))
        )
        if prod[-1] == '1':
            vch1 = vch_prod.clone()
        else:
            vch1 = pl.concat([vch1, vch_prod])

    # voucher_mapping = {'LI1': 4, 'LI2': 5, 'LI3': 3, 'LIV': 2}
    # vch1 = vch1.with_columns(pl.col('PROD_CD').replace(voucher_mapping,return_dtype=pl.Int64).alias('product_id')).fill_null(0)#.drop('PROD_CD')
    vch1 = vch1.fill_null(0)

    return(vch1)

In [6]:
def get_summed_period_iid_metric(metric,prod_cd):
    columns = ['IID','PROD_CD'] + [metric+str(i) for i in range(1,106)]
    df = pl.read_parquet(xpn+'LAX.parquet',columns=columns).filter(pl.col('PROD_CD').is_in(prod_cd))

    # 1,4,13,26 for current and prior period for a given Metric
    df = df.select(
        pl.col('IID'),pl.col('PROD_CD'),
        pl.col(metric+'1').alias(metric+'_1c'),
        pl.sum_horizontal([metric+str(i) for i in range(1,5)]).alias(metric+'_4c'),
        pl.sum_horizontal([metric+str(i) for i in range(1,14)]).alias(metric+'_13c'),
        pl.sum_horizontal([metric+str(i) for i in range(1,27)]).alias(metric+'_26c'),
        pl.sum_horizontal([metric+str(i) for i in range(1,num_weeks_rx+1)]).alias(metric+'_qtdc'),

        pl.col(metric+'2').alias(metric+'_1p'),
        pl.sum_horizontal([metric+str(i) for i in range(5,9)]).alias(metric+'_4p'),
        pl.sum_horizontal([metric+str(i) for i in range(14,27)]).alias(metric+'_13p'),
        pl.sum_horizontal([metric+str(i) for i in range(27,53)]).alias(metric+'_26p'),
        pl.sum_horizontal([metric+str(i) for i in range(14,14+num_weeks_rx)]).alias(metric+'_qtdp'),

        pl.sum_horizontal([metric+str(i) for i in range(1,106)]).alias(metric+'_all')
    )
    # For Voucher Removal - 
    if metric == 'TUF':
        dfv = get_lin_voucher()
        df = df.join(dfv,on=['IID','PROD_CD'],how='left').fill_null(0)
        cols_to_remove = dfv.columns[1:-1]
        df = df.with_columns(
            pl.col(f'{metric}_1c') -  pl.col(f'v{metric}_1c').alias(f'{metric}_1c'),
            pl.col(f'{metric}_4c') -  pl.col(f'v{metric}_4c').alias(f'{metric}_4c'),
            pl.col(f'{metric}_13c') -  pl.col(f'v{metric}_13c').alias(f'{metric}_13c'),
            pl.col(f'{metric}_26c') -  pl.col(f'v{metric}_26c').alias(f'{metric}_26c'),
            pl.col(f'{metric}_qtdc') -  pl.col(f'v{metric}_qtdc').alias(f'{metric}_qtdc'),
            pl.col(f'{metric}_1p') -  pl.col(f'v{metric}_1p').alias(f'{metric}_1p'),
            pl.col(f'{metric}_4p') -  pl.col(f'v{metric}_4p').alias(f'{metric}_4p'),
            pl.col(f'{metric}_13p') -  pl.col(f'v{metric}_13p').alias(f'{metric}_13p'),
            pl.col(f'{metric}_26p') -  pl.col(f'v{metric}_26p').alias(f'{metric}_26p'),
            pl.col(f'{metric}_qtdp') -  pl.col(f'v{metric}_qtdp').alias(f'{metric}_qtdp'),
            pl.col(f'{metric}_all') -  pl.col(f'v{metric}_all').alias(f'{metric}_all')
        ).drop(cols_to_remove)

    # Adding MP related columns
    df = df.join(mp_spec_seg_dec,on='IID',how='left').filter(pl.col('geography_id').is_not_null())

    return(df.drop(['specialty_group','segment','decile','geography_id']))

In [7]:
# Xponent_Calls Creation - 
all_products_tuf = get_summed_period_iid_metric('TUF',['LI1','LI2','LI3'])
all_products_nuf = get_summed_period_iid_metric('NUF',['LI1','LI2','LI3'])

def add_parent_product_rows(df):
    agg_dict = {}
    for col in df.columns[2:]:
        agg_dict[col] = pl.col(col).sum()
    
    #join_cols = ['geography_id','plan_type','PlanID','IID']

    df = df.join(prod_mapping[['code','product_id','parent_product_id']], left_on = 'PROD_CD',right_on = 'code', how = 'left')
    df_2_35 = df.filter(pl.col('parent_product_id').is_in([2,35]))
    df_2_35 = df_2_35.group_by(['IID','parent_product_id']).agg(**agg_dict).rename({'parent_product_id':'product_id'})
    
    df_1 = df.group_by('IID').agg(**agg_dict).with_columns(product_id = pl.lit(1)).with_columns(pl.col('product_id').cast(pl.Int64))

    # stack 1, 2_35 with df and return
    df = df.drop(['PROD_CD','parent_product_id']) #dropping to make same shape
    vstack_helper = df.columns
    df = df.vstack(
        df_2_35.select(vstack_helper)
    ).vstack(
        df_1.select(vstack_helper)
    )

    return(df)

all_products_tuf = add_parent_product_rows(all_products_tuf)
all_products_nuf = add_parent_product_rows(all_products_nuf)

tuf1 = all_products_tuf.filter(pl.col('TUF_all')!=0).select(['IID','product_id'])
nuf1 = all_products_nuf.filter(pl.col('NUF_all')!=0).select(['IID','product_id'])
xponent = tuf1.join(nuf1,on=['IID','product_id'],how='outer_coalesce')

calls = (
    temp_calls.filter(pl.col('call_week')<= num_weeks_calls)
    .rename({'AttendeeIID':'IID'})
    .select('IID').unique('IID')
    .with_columns(product_id = pl.lit(2)).with_columns(pl.col('product_id').cast(pl.Int64))
)

xponent_calls = xponent.join(calls,on=['IID','product_id'],how='outer_coalesce')

#delete extra dfs when optimizing

---

In [8]:
# Processing -
temp1 = (
    MASTER_UNI.select(
        [
            'IID','FirstName','LastName','CREDENTIAL',
            'AddressLine1','AddressLine2','AddressLine3','AddressLine4','CityName','StateCode','ZIP','DECILE'
        ]
    )
    .with_columns(
        pl.concat_str([pl.col('AddressLine1'),pl.col('AddressLine2'),pl.col('AddressLine3'),pl.col('AddressLine4')],separator=' ',ignore_nulls=True).alias('Address'),
        pl.concat_str([pl.col('LastName'),pl.col('FirstName')],separator=', ',ignore_nulls=True).alias('Physician_Name')
    )
    .join(mp_spec_seg_dec,on='IID',how='left')
    .join(ibsc_ptype,on='IID',how='left')
    .drop(['AddressLine1','AddressLine2','AddressLine3','AddressLine4','FirstName','LastName','decile'])
)

In [9]:
# For Filtering - 
xponent_calls_1 = xponent_calls.select(['IID']).unique().with_columns(fl1 = pl.lit(1))

temp1_test = temp1.join(xponent_calls_1,on='IID',how='left')

temp1_test = temp1_test.with_columns(
    pl.when(
        (pl.col('segment') == 'Target') | (pl.col('specialty_group') == 'PED')
    ).then(pl.lit(1)).otherwise(pl.lit(None)).alias('fl2')
)
# Filter - 
temp1_test = temp1_test.filter(
    (pl.col('fl1').is_not_null()) | (pl.col('fl2').is_not_null())
)

temp1 = temp1_test.drop(['fl1','fl2'])

In [10]:
# Converting to feed ready data
#Renaming columns
column_mapping = {
    "IID": "Physician_ID",
    "CREDENTIAL": "Credential",
    "CityName": "City",
    "StateCode": "State",
    "ZIP": "Zip",
    "DECILE": "Decile",
    "Address": "Address",
    "Physician_Name": "Physician_Name",
    "specialty_group": "Specialty",
    "segment": "Segment",
    "geography_id": "Geography_id",
    "IBSC_VALUE": "IBSCPrimaryPayerType"
}
final_feed = temp1.rename(column_mapping)

# changing value of column to match with sas - 06/21
final_feed = final_feed.with_columns(
        pl.when(pl.col('Segment')=='ALG-ONLY-TARGET')
        .then(pl.lit('AGNT'))
        .when(pl.col('Segment')=='Target')
        .then(pl.lit('T'))
        .when(pl.col('Segment')=='Non-Target')
        .then(pl.lit('NT'))
        .alias('Segment'))

# arranging columns according to feed
req_cols = ["Physician_Name", "Physician_ID", "Geography_id", "Segment", "Specialty", "Decile", "Address", "City", "State", "Zip", "IBSCPrimaryPayerType", "Credential"]
final_feed = final_feed.select(req_cols)#final_dataset
final_feed = final_feed.with_columns(pl.col('IBSCPrimaryPayerType').fill_null('N/A'))

In [11]:
#Exporting Feeds-
OUT = 's3://vortex-staging-a65ced90/BIT/output/LinzessSnapshot/Weekly/'
final_feed.to_pandas().to_csv(f'{OUT}Weekly_LinzessSnapshot_ProfileInfo_Feed.txt', sep='|',lineterminator='\r\n',index=False)
print('LS Profile Info Exported !')

LS Profile Info Exported !
